<font size="6">Project Name</font>
<br>
<font size="4">Project Description. </font>
<br>
<br>
<font size="4">Business Tasks:</font>
<font size="3">
1. Business Task 1
1. Business Task 2
1. Business Task 3 </font>

In [ ]:
pip install ipython-sql

In [ ]:
%load_ext sql

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import sqlite3

<font size="6">Data Preparation</font>

<font size="3">Convert excel sheet(s) or CSV(s) to SQL tables</font>

In [ ]:
# Directory containing your XLSX files
xlsx_directory = '/Users/leahbowers/Portfolio/[Project Folder]/'

# SQLite database file
db_file = 'project_name.db'

# Create a SQLite database connection
conn = sqlite3.connect(db_file)

# Iterate through XLSX files in the directory
for excel_file in os.listdir(xlsx_directory):
    if excel_file.endswith('.xlsx'):
        file_path = os.path.join(xlsx_directory, excel_file)
        
        # Read the XLSX file into a DataFrame
        df = pd.read_excel(file_path)
        
        # Replace spaces and special characters in column names with underscores
        df.columns = [c.replace(' ', '_').replace('.', '_').replace('-', '_') for c in df.columns]
        
        # Determine the table name (use the file name without extension)
        table_name = os.path.splitext(excel_file)[0]
        
        # Write the DataFrame to the SQLite database as a new table
        df.to_sql(table_name, conn, if_exists='replace', index=False)

# Commit changes and close the database connection
conn.commit()
conn.close()

In [ ]:
# Directory containing your CSV files
csv_directory = '/Users/leahbowers/Portfolio/[Project Folder]/'

# SQLite database file
db_file = 'project_name.db'

# Create a SQLite database connection
conn = sqlite3.connect(db_file)

# Iterate through CSV files in the directory
for csv_file in os.listdir(csv_directory):
    if csv_file.endswith('.csv'):
        file_path = os.path.join(csv_directory, csv_file)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Replace spaces and special characters in column names with underscores
        df.columns = [c.replace(' ', '_').replace('.', '_').replace('-', '_') for c in df.columns]
        
        # Determine the table name (use the file name without extension)
        table_name = os.path.splitext(excel_file)[0]
        
        # Write the DataFrame to the SQLite database as a new table
        df.to_sql(table_name, conn, if_exists='replace', index=False)

# Commit changes and close the database connection
conn.commit()
conn.close()

In [ ]:
# Connect to Project database
%sql sqlite:///project_name.db

In [ ]:
# Check that table was converted from csv and pulled into database
conn = sqlite3.connect('project_name.db')
query = """
    SELECT 
        name 
    FROM sqlite_master 
    WHERE type='table';
"""
result = pd.read_sql_query(query, conn)
print(tabulate(result, headers='keys', tablefmt='psql', showindex=False))

<font size="3">**Optional:** Cleanup database by removing unwanted tables</font>

In [ ]:
#Clean up project database
conn = sqlite3.connect('database_name.db')

cursor = conn.cursor()
table_name_to_remove = 'unwanted_table_name'

drop_table_query = f"DROP TABLE IF EXISTS {table_name_to_remove};"
cursor.execute(drop_table_query)

conn.commit()
conn.close()

<font size="6">Preview Tables</font>

In [ ]:
# Confirm data types are correct
conn = sqlite3.connect('database_name.db')
query = """
    PRAGMA table_info(table_name)
"""
result = pd.read_sql_query(query, conn)
print(tabulate(result, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Get preview of project table
pd.set_option('display.max_columns',None)
table_name = pd.read_csv('/Path/to/CSV/File/file_name.csv', encoding='latin1')
table_name.head(5)

<font size="6">Data Cleaning</font>

<font size="3">Find duplicates and missing values</font>

In [ ]:
# Check for duplicates in the entire table
duplicate_rows = table_name[table_name.duplicated()]
if duplicate_rows.empty:
    print("No duplicate rows found.")
else:
    print("Duplicate Rows:")
    print(duplicate_rows)
    
# Check for missing values in the entire table
nan_values = table_name.isna().sum()
if nan_values.sum() == 0:
    print("No missing values found.")
else:
    print("NaN (Missing Values) Count:")
    print(nan_values)

<font size="3">Remove duplicate rows and replace missing values</font>

In [ ]:
# If there are duplicates, remove duplicate rows based on all columns
table_name_clean = table_name.drop_duplicates()

# If no duplicates, start here. Replace blank cells with 0 in specific columns. 
table_name_clean = table_name["column1"].fillna(0, inplace=True)

# Replace blank cells with "not given" in specific column.
table_name_clean = table_name["column2"].fillna("not given", inplace=True)

# Check for missing values in the entire table
nan_values = table_name_clean.isna().sum()
if nan_values.sum() == 0:
    print("No missing values found.")
else:
    print("NaN (Missing Values) Count:")
    print(nan_values)

<font size="3">Remove whitespace</font>

In [ ]:
for column in table_name.columns:
    # Check if the column contains strings (object dtype)
    if table_name[column].dtype == 'object':
        # Strip leading and trailing whitespace from string values
        table_name[column] = table_name[column].str.strip()

<font size="3">Rename Columns</font>

In [ ]:
table_name_clean.rename(columns={"column_name":"renamed_column","column_name2": "renamed_column2"}, inplace=True)
print(table_name_clean.columns)

<font size="3">Concatenate columns and convert to different data type</font>

In [ ]:
# Concatenate multiple columns into a single string column with delimiter
table_name_clean['new_column'] = table_name_clean['column1'].astype(str) + '[delimiter]' + table_name['column2'].astype(str)

# Convert the concatenated column to different data type format
table_name_clean['new_column'] = pd.to_[datatype](table_name_clean['new_colun'], format='%m/%d/%Y')

# Drop the individual component columns if needed
table_name_clean.drop(columns=['column1', 'column2'], inplace=True)

table_name_clean.head(5)

<font size="3">Apply proper case</font>

In [ ]:
# Apply the title case transformation to column1
table_name_clean['column1'] = table_name_clean['column1'].str.title()

table_name_clean.head(5)

<font size="3">Split column with multiple parts and reorder</font>

In [ ]:
# Find the maximum number of parts in the column with multiple parts. Add 1 to account for columns with only 1 part
max_parts = table_name_clean['column with mult. parts'].str.count(',') + 1 

# Determine the maximum number of parts
max_n = max_parts.max()

# Split the new_column by comma and expand it into separate columns
split_columns = [f'new_column{i}' for i in range(1, max_n + 1)]
split_artists = table_name_clean['new_column'].str.split(',', expand=True, n=max_n)
split_artists.columns = split_columns

# Drop the original column if you don't need it anymore
table_name_clean.drop(columns=['column with mult. parts'], inplace=True)

print(table_name_clean.columns)

<font size="3">Reorder columns</font>

In [ ]:
#List new column names in desired order
column_order = ['column1', 'column2', 'column3']
table_name_clean = table_name_clean[column_order]

table_name_clean.head(5)

<font size="3">Remove non-character symbols from strings</font>

In [ ]:
# Remove symbols from column1
table_name_clean['column1'] = table_name_clean['column1'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)

# Display the updated DataFrame
table_name_clean.head(15)

<font size="3">Print cleaned table to CSV</font>

In [ ]:
# Specify the path where you want to save the CSV file
csv_file_path = '/Path/to/CSV/File/table_name_clean.csv'

# Save the DataFrame to the CSV file
table_name_clean.to_csv(csv_file_path, index=False)

# Print a message to confirm the file has been saved
print(f"DataFrame saved to {csv_file_path}")

<font size="6">Analysis</font>

In [ ]:
#Get preview of online_sales table
pd.set_option('display.max_columns',None)
[var] = pd.read_csv('/Users/leahbowers/Portfolio/[Project Folder]/project_name.csv')
var.head(5)

In [ ]:
# Check for duplicates
duplicate_rows = var[var.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

# Check for NaN (missing values)
nan_values = var.isna().sum()
print("\nNaN (Missing Values) Count:")
print(tabulate(result, headers='keys', tablefmt='psql', showindex=False))

<font size="6">Question 1</font>

<font size="3">**Question**</font>

<font size="3">Answer</font>

In [ ]:
#Get total revenue across store
conn = sqlite3.connect('project_name.db')
query = """
QUERY;
"""
result = pd.read_sql_query(query, conn)
print(tabulate(result, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
conn = sqlite3.connect('online_sales.db')
query = """
QUERY;
"""
result = pd.read_sql_query(query, conn)

month_names = {
    '01': 'Jan',
    '02': 'Feb',
    '03': 'Mar',
    '04': 'Apr',
    '05': 'May',
    '06': 'Jun',
    '07': 'Jul',
    '08': 'Aug',
    '09': 'Sep',
    '10': 'Oct',
    '11': 'Nov',
    '12': 'Dec'
}
result['month'] = result['month'].map(month_names)

plt.figure(figsize=(8, 4))
plt.bar(result['month'], result['total_revenue'], color=['#fe938c', '#edaf97', '#c49792', '#ad91a3'])
plt.title('Total Revenue by Month Shows Strongest Earnings in January and Weakest in February', fontsize = 14)
plt.xlabel('Month',fontsize = 12)
plt.ylabel('Total Revenue',fontsize = 12)

In [ ]:
ax = sales['browser'].value_counts(normalize=True).plot(
    kind='pie',
    autopct='%1.1f%%',
    ylabel=' ',
    shadow=True,
    colors=['#fe938c', '#edaf97', '#c49792', '#ad91a3'],
    fontsize=12
)

ax.set_title('Chrome is the Preferred Browser for Et Femme Shoppers', fontsize=14)
ax.legend(loc='upper right')
plt.show()

<font size="6">Question 2</font>

<font size="3">**Question**</font>

<font size="3">Answer</font>

<font size="6"> Question 3 </font>

<font size="3">**Question**</font>

<font size="3">Answer</font>

<font size="6">Conclusion and Recommendations</font>

<font size="3"> Description.</font>

<font size="2">All data for this project was acquired through Kaggle's Database "[Database]"</font>